## Programming Assignment 4 - Kushlevitz and Mansour Algorithm

Fill in your details in the below cell. \
Your Name :  Soham Chatterjee\
ID : BMC202175

Rename the notebook with your ID as a suffix before submitting in the moodle page.

### Randomized Algorithms
Randomized algorithms and probabilistic methods play a key role in modern computer science. Randomized algorithms make random choices during their execution and achieve results with certain accuracy within a reasonable time. In today's exercise, we are going to see how randomized algorithms can be efficient as compared to their brute-force counterparts using a well-known algorithm to learn boolean functions called "Kushlevitz-Mansour" (KM) algorithm.

This is a primer for the classes starting from Tuesday i.e., Nov 7 where we will be looking at how to exploit boolean functions, fourier analysis in both classical and quantum to develop new ideas for Quantum Algorithmic design.

### Background
* Consider a boolean function of three inputs $x_1,x_2,x_3 \in \{-1,+1\}$ whose output is determined by the expression $f = 0.5 + 0.5x_2 + 0.5x_3 - 0.5x_2x_3$.
* In general, for a 3-variable boolean function, there are 8 (2^3) possible terms $(\phi, x_1, x_2, x_3, x_1x_2, x_1x_3, x_2x_3, x_1x_2x_3)$. But for the above case we can see that only 4 monomial terms are present. 
* These coefficients are called as fourier coefficients and the set of these coefficients is called as a fourier spectrum.
* Now, consider learning the expression when the number of variables is large, say n=200. It will require a truth table of entries 2^200 which is impossible to store in the memory let alone computing the spectrum in a brute-force approach.
* This is where randomized algorithms like Kushlevitz-Mansour come to the rescue.
* KM algorithm outputs an expression which is close to the original function, in polynomial time, given the set of fourier coefficients is sparse. Such functions are usually referred to as *concentrated* boolean functions.

## Kushlevitz - Mansour Algorithm 
* References:
  1. LEARNING DECISION TREES USING THE FOURIER SPECTRUM https://dl.acm.org/doi/pdf/10.1145/103418.103466 
  2. Implementation Issues in the Fourier Transform Algorithm: https://link.springer.com/article/10.1023/A:1011034100370 
  
* Assumptions: 
  1. Query access to an oracle to the target function must be available to query it for a given input.
  2. Fourier spectra of the function is concentrated.

* Principle: The approach works by iteratively growing the strings corresponding to the monimial terms (prefix), estimate the corresponding fourier coefficient and thereafter prune the strings whose fourier coefficient is not larger than a given *threshold*. The pruning allows us to significantly reduce the number of monimial terms to be explored without comprimising on the final accuracy.

**Note: This algorithm is equivalently applicable for both Boolean Valued Boolean Function as well as Real Valued Boolean Function**

## Algorithm 
* Important parameters: $n, \alpha, k, m_1, m_2, \theta, \delta$
    * n - Problem size in terms of input variables. 
    * $\theta$ and $\delta$ are threshold parameters. We can set these values to as per the required accuracy and probabilistic guarantee. Smaller the threshold, higher the accuracy however has increased execution time. So, there is always a trade-off.
    * m1 and m2 are repetition parameters. How many number of random inputs to be selected to compute significance of an alpha value.
        * $ m1 = floor((1/\theta^2)*log2(n/(\delta*\theta^2)))$ 
        *  $m2 = floor((1/\theta^4)*log2(n*m1/(\delta*\theta^2)))$
    *  Experimentally, values of m1 and m2 are found to be good even when restricted as m1 <=100 and m2 <= $5/(\theta^2)$ [2]
    *  k = Divide input bit string into two parts of size k and (n-k). Start from k = 1 and increment k by 1 upto n
    *  $\alpha$ = element in the set S, where S = {Set of all those input combinations of k bits for which $\beta_\alpha \geq$  threshold = $\theta^2/2$}
        * For k=1, $\alpha = [0,1]$
        * For each $\alpha$

The algorithm can be explained using two subroutines Coeff($\alpha$) and Approx($\alpha$) as follows\
SUBROUTINE Coeff($\alpha$)
* $\beta_\alpha$ = Approx($\alpha$)  // $\beta_\alpha$ estimates the total value of fourier coefficients that share the prefix with $\alpha$ 
    * IF $\beta_\alpha \geq \theta^2/2$ THEN
        * IF $|\alpha| == n$ THEN Output $\alpha$
    * ELSE Coeff($\alpha 0$); Coeff($\alpha 1$) // Explore

Consider for a $l$-bit binary string $\alpha \in \{0,1\}^l$, the corresponding monomial $\chi_{\alpha} = \prod_{i}x_i^{\alpha[i]}$ and the value $\chi_{\alpha}(z) = (-1)^{\alpha.z}$ for any $z \in \{0,1\}^l$.\
SUBROUTINE Approx($\alpha$):
* Choose at random $x_i \in \{0,1\}^{n-k}$ for $1\leq i\leq m_1$
* For each $x_i$
    * Choose at random $y_{i,j} \in \{0,1\}^{k}$ for $1\leq j \leq m_2$
        * Let $A_i = \frac{1}{m_2}\sum\limits_{j=1}^{m_2}f(y_{i,j}x_i)\chi_{\alpha}(y_{i,j})$
* Compute $\beta_\alpha = \frac{1}{m_1}\sum\limits_{i=1}^{m_1}A_i^2$
* Output $\beta_\alpha$

### Input problem
For a boolean function given below, Implement Kushlevitz and Mansour Algorithm to find best approximation of $f$ using the largest fourier coefficients. Also compare this spectrum with the fourier spectrum obtained using Brute-Force Method.
* Consider the function $f : \{-1,1\}^3 \rightarrow \{-1,+1\}$ described above whose truth table can be given as:
<div align="center">

| x | f(x) |
| :---: | :---: | 
| 1,1,1 | 1 |
| 1,1,-1 | 1 |
| 1,-1,1 | 1 |
| 1,-1,-1 | -1 |
| -1,1,1 | 1 |
| -1,1,-1 | 1 |
| -1,-1,1 | 1 |
| -1,-1,-1 | -1 |

</div>

Throughout the rest of this exercise, we will use the domain of boolean variables to be either $\{0,1\}$ or $\{-1,+1\}$. Clearly, one can be transformed to another through change of basis.

## Output
Find the fourier expansion of the majority function using KM algorithm

## Code Implementation

### Import all the necessary libraries numpy, math, random, time

In [1]:
# Import all the necessary libraries numpy, math, random, time
import numpy as np                                    
import math
import random
import time
seed = np.random.seed(0)

### 1. Define parameters

In [2]:
"""
 Define parameters
 n - Problem size in terms of input variables. Change it as per the problem size
 theta,delta,m1,m2 - Parameters defined in the algorithm
"""
n = 3              
theta = 0.5                # Theta and delta parameters, feel free to change
delta = 0.1

# Expression for m1 and m2

m1 = int((1/theta**2)*math.log2(n/(delta*theta**2))) 
m2 = int((1/theta**4)*math.log2(n*m1/(delta*theta**2)))


"""
    Values of m1 and m2 are usually restricted to be m1<=100 and m2 ,= 5/(theta**2)
"""

if(m1>=100):
    m1 = 100
m2_max = math.ceil(5/(theta**2))
if(m2>=m2_max):
    m2 = m2_max

print(m1,m2)

27 20


### 2. Define access to the function

In [3]:
def f_val(x):
    f = [1,1,1,-1,1,1,1,-1]
    return f[x]

### 3. Helper Functions
To make the final implementation more modular, following two function definitions are added which can be used as needed in the final program.
1. generate_binary_string(l): Function to generate a random bit string of length l. This is helpful while estimating the value of $\beta_\alpha$

2. dot(a,b): Function to compute the dot product of two binary strings $a,b$ given as $\sum a[i].b[i]$. During the estimation of $\beta_\alpha$ we see that $\chi_\alpha(y_{i,j})$ is computed repeatedly.

In [4]:
"""
    -Function to generate a random binary string of size l bits
"""
def generate_binary_string(l):
    bin_string = []
    for i in range(l):
        bin_string.append(random.randint(0,1))
    
    return bin_string

In [5]:
def bin_to_dec(b):
    n = 0
    for i in range(len(b)):
        n = n + (2**(i))*(b[i])
    return n

In [6]:
def exponent(a,b):
    if a == 0:
        return 1
    elif a == 1:
        return b
    elif a % 2 == 0:
        n = exponent(a//2 , b)
        return n*n
    else:
        n = exponent(a//2, b)
        return (n*n)*b

In [7]:
def dot(a,b):
    length = len(a)
    norm = 0
    for i in range(length):
        norm = norm + a[i]*b[i]
        
    return norm


### 4. Two main subroutines of the algorithm

In [8]:
"""
    - SUBROUTINE Approx(alpha,k)
    - Returns an estimate of the significance of the fourier coefficient whose prefix is alpha, where length of alpha is k. 
"""
def approx(alpha, k):
    """
        - Special case: k==n; # for k==n case, there is no list of x_i s
        - Generate yij_list. This has two cases, 2**(k)<=m2 and 2**(k)>m2. We chose the length of yij to be min of either. 
        - Compute A_i by iterating of yij and beta_alpha (Recall that there is only one i in this case)
    """
    min1 = min(m1, exponent(n-k,2))
    min2 = min(m2, exponent(k,2))
    if(k==n):
        ## Write your code here for the case when k==n
        yi = []
        for j in range(min2):
            yij = generate_binary_string(n)
            yi.append(yij)
        A_i = 0
        for j in range(min2):
            # value of sum f(yij) xhi_alpha(yij)
            A_i = (f_val(bin_to_dec(yi[j])))*((-1)**(dot(alpha, yi[j]))) + A_i 
        A_i = A_i/min2 # approximated value of f_alpha(xi)
        return [A_i**2, A_i] # keeps track of the expected fourier coefficient when n==k
    else:
        """
            1. Generate xi_list. This has two cases, 2**(n-k)<=m1 and 2**(n-k)>m1. We chose the length of xi to be min of either.
            2. Generate yij_list. This has two cases, 2**(k)<=m2 and 2**(k)>m2. We chose the length of yij to be min of either.
            3. Iterate over x_i and y_ij and compute A_i and finally beta_alpha
        """
        # Part-1: Write a code to Generate random Xi string
        beta_alpha = 0
        A = []
        for i in range(min1):
            xi = generate_binary_string(n-k)
            yi = []
        # -----------------------------------------------------------------------
        # Part -2 :  Write a code to Generate random Yi string
            for j in range(min2):
                yij = generate_binary_string(k)
                yi.append(yij)
            
        #------------------------------------------------------------
        # Part 3: Iterate over xi and yj and compute Ai and beta_alpha
            A_i = 0
            for j in range(min2):
                # value of sum f(yij xi) xhi_alpha(yij)
                A_i = (f_val(bin_to_dec(xi+yi[j])))*((-1)**(dot(alpha, yi[j]))) + A_i
            A_i = A_i/min2 # value of f_alpha(xi)
            A.append(A_i)
        for i in range(min1):
            beta_alpha = beta_alpha + (A[i]**2)
        beta_alpha = beta_alpha/min1
        return [beta_alpha, beta_alpha] # keeps track of the expected fourier coefficient

In [9]:
"""
    SUBROUTINE Coef(alpha,k)
    Input: alpha value, k - Size of alpha
    Output: Set of binary strings corresponding to the monomials with largest fourier coefficients 
    This function is called recursively call till k==n by extending the prefix with both '0' and '1' 
    and pruned as needed (refer to the algorithm).
"""

def Coef(alpha,k, list):
    # Compute B_alpha for each value of alpha and keep those alpha values for which B_alpha> threshold
    # Write your code here
    B_alpha_list = approx(alpha, k)
    if B_alpha_list[0] >= (theta**2)/2:
        if k == n:  
            list.append([alpha, B_alpha_list[1]])  
        else:
            Coef(alpha+[0], k+1, list)   
            Coef(alpha+[1], k+1, list)
    
            

### Call to SUBROUTINE Coef(alpha)

In [10]:
if __name__ == '__main__':
    k=1
    alpha = [0]
    coephs = []
    Coef([], 0, coephs)  # starting with 0 then we will recurse on adding new bits
    print(coephs)

[[[0, 1, 0], 0.5], [[0, 1, 1], 0.75], [[1, 0, 1], -0.5], [[1, 1, 0], -0.75]]


In [13]:
# Using the alpha values and the corresponding fourier estimates, create the fourier expansion of the majority function
# You can output the expression as a string and print below
def bin_to_coefficient(lst):
    strg = ""
    flag = True
    for i in range(len(lst)):
        if lst[i] == 1:
            strg = strg + "x_" + str(len(lst)-i)
            flag = False
    if flag:
        strg = strg + '1'
        return strg
    return strg
to_print = []       
for coefficient in coephs:
    if coefficient[1] < 0:
        to_print.append('(')
        to_print.append(str(coefficient[1]))
        to_print.append(')')
    else:
        to_print.append(str(coefficient[1]))
        
    to_print.append('*')
    to_print.append(bin_to_coefficient(coefficient[0]))
    to_print.append(' + ')
for item in to_print[:-1]:
    print(item, end="")

0.5*x_2 + 0.75*x_2x_1 + (-0.5)*x_3x_1 + (-0.75)*x_3x_2

For interested students, below are some more examples of the concentrated functions with increasing problem size. You can see that the efficiency of the KM algorithm is more pronounced as the problem size increases. 

Also provided insights to how we can further improve the speed of the algorithm further using parallel processing ability of modern day processors.

__Please note that the below exercises are not part of the evaluation__

## Optional 1: K-Junta Function
* A function $f:\{-1,1\}^n \rightarrow \{-1,1\}$  is called a k-junta function for $k < n$, if the output strictly depends on just $k$ inputs.
* You can verify that the function we solved above is $2$-junta
* Here we generate a random function of 5 variables and repeat the same output for $2^{15}$ times which shall give us a $5$-junta function of input size $200$

In [ ]:
"""
    Ex. 5: Generate a random function f
"""  
n = 5
N = 2**n
f = []
for i in range(N):
    f.append((-1)**np.random.randint(0,2))
#print(f)
#for i in range(len(f)):
#    f[i] = (-1)**np.random.randint(0,2)
print(f)
def f_val(a_int):
    a = a_int % N
    return f[a]

## Optional 2: Multithreading and Multiprocessing are important coding practices to use the available computing power of your machine and reduce the execution time. It implements the parallel processing mechanism using all the cpu cores of your machines.
* Two alternatives for parallel processing are 1. Multi-threading 2. Multi-Processing
* However, Multi-threading is not possible with this algorithm....find out why?
* Multi-processing is possible. Hint: p = multiprocessing.Pool() and p.map()
* A problem of size n=200 variables, with m1 = 100 and m2 = 500 repetitions, takes in general 11 Hours to complete the execution
* With multi-processing, this time is reduced to 2 Hours on the same machine using all its available 16 cores of the processor

## Optional 3: Random functions with large number of inputs

In [ ]:
# Generate a random function. One example is shown below
#f = (x32 © x8 © x34 © x13) AND (x33 © x20 © x9 © x1 © x15 © x39) OR (x24 © x10 © x27 © x6 © x34)
# f = f1 and f2 or f3
n = 40

# Hardcode the f1,f2,f3 lists as 
f1_list = [32,8,34,13]
f2_list = [33,20,9,1,15,39]
f3_list = [24,10,27,6,34]
f_list = set(f1_list + f2_list + f3_list )

# OR Generate f1_list, f2_list, f3_list randomly as below

f1_l1 = 7
f2_l2 = 5
f3_l3 = 8
f4_l4 = 4
f1_list,f2_list,f3_list,f4_list = [],[],[],[]
f = []
i=0
while(i<f1_l1):
    r=random.randint(1,n)
    if r not in f1_list:
        f1_list.append(r)
        i = i+1
    else:
        pass

# Write similar code for f2_l2, f3_l3, f4_l4
f_list = set(f1_list + f2_list + f3_list + f4_list)
print(f' f1_list data is {f1_list}')
print(f' f2_list data is {f2_list}')
print(f' f3_list data is {f3_list}')
print(f' f4_list data is {f4_list}')
print(f' f_list data is {f_list}')


def count(f_bits):
    f_ = f_bits.count(1)
    if (f_%2==0):
        return 0
    else:
        return 1
    
    
def f_val(a_int):
    a = bin(a_int)[2:].zfill(n)
    f1_bits = []
    f2_bits = []
    f3_bits = []
    f4_bits = []
    for i in f1_list:
        f1_bits.append(int(a[n-i]))
    for i in f2_list:
        f2_bits.append(int(a[n-i]))
    for i in f3_list:
        f3_bits.append(int(a[n-i]))
    for i in f4_list:
        f4_bits.append(int(a[n-i]))
    #print(f1_bits)
    #print(f2_bits)
    #print(f3_bits)
    #print(f4_bits)
    f1 = count(f1_bits)
    f2 = count(f2_bits)
    f3 = count(f3_bits)
    f4 = count(f4_bits)
    f = f1 and f2 or f3

    return (-1)**f

In [ ]:
# Write a code to compute fourier coefficients and final alpha values as explained

In [ ]:
# Verify your end result for following 3 points by compairing it with elements present in the original function 
# 1. Check for Elements correctly retrieved
# 2. Elements retrieved but not present in the original function 
# 3. Elements present in the original function but not retrieved

# Write your code here

In [ ]:
# Compare total execution times of both and accuracy of the KM

## Optional 4: Examples for further practice

In [ ]:
"""
    Ex.3 (4 variables), Ex. Ref. 1 sec. 2.3.1  page no 8
    f(x1,x2,x3,x4) = x1 AND x3 (x1 is lsb and x4 is msb, x=x4, y=x3, z=x2, w=x1)
    f = [1,1,1,1,1,-1,1,-1,1,1,1,1,1,-1,1,-1]
"""
def f_val(a_int):
    a = bin(a_int)[2:].zfill(n)
    f = int(a[1]) and int(a[3])
    return (-1)**f

In [ ]:
"""
    Ex. 4 (40 variables), Ref. 1, sec. 3.1 page no 15
    f = (x32 © x8 © x34 © x13) AND (x33 © x20 © x9 © x1 © x15 © x39) OR (x24 © x10 © x27 © x6 © x34)
          
"""
def f_val(a_int):
    a = bin(a_int)[2:].zfill(n)
    f1 = int(a[40-32])^int(a[40-8])^int(a[40-34])^int(a[40-13])
    f2 = int(a[40-33])^int(a[40-20])^int(a[40-9])^int(a[40-39])
    f3 = int(a[40-24])^int(a[40-10])^int(a[40-27])^int(a[40-34])
    f = f1 and f2 or f3

    return (-1)**f